In [1]:
import os
import tensorflow
os.environ['KERAS_BACKEND'] = 'tensorflow'

import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import pandas as pd
from termcolor import colored

# Load data
print(colored("Loading train and test data", "green"))
train_data = pd.read_csv('twt_#ifc.csv')
test_data = pd.read_csv('nit1.csv')
print(colored("Data loaded", "green"))

Loading train and test data
Data loaded


Using TensorFlow backend.


In [2]:
# Tokenization
print(colored("Tokenizing and padding data", "green"))
tokenizer = Tokenizer(num_words = 2000, split = ' ')
tokenizer.fit_on_texts(train_data['full_text'].astype(str).values)
train_tweets = tokenizer.texts_to_sequences(train_data['full_text'].astype(str).values)
max_len = max([len(i) for i in train_tweets])
train_tweets = pad_sequences(train_tweets, maxlen = max_len)
test_tweets = tokenizer.texts_to_sequences(test_data['full_text'].astype(str).values)
test_tweets = pad_sequences(test_tweets, maxlen = max_len)
print(colored("Tokenizing and padding complete", "yellow"))


Tokenizing and padding data
Tokenizing and padding complete


In [3]:

# Building the model
print(colored("Creating the LSTM model", "yellow"))
model = Sequential()
model.add(Embedding(2000, 128, input_length = train_tweets.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(256, dropout = 0.2))
model.add(Dense(3, activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Creating the LSTM model
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 34, 128)           256000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 34, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               394240    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 771       
Total params: 651,011
Trainable params: 651,011
Non-trainable params: 0
_________________________________________________________________


In [4]:
# Training the model
print(colored("Training the LSTM model", "green"))
history = model.fit(train_tweets, pd.get_dummies(train_data['sent_score']).values, epochs = 10, batch_size = 128)
print(colored(history, "green"))


Training the LSTM model


C:\Users\TEMP.DESKTOP-8R6RI8B.000\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
1749/1749 [==============================] - 7s 4ms/step - loss: 1.0200 - accuracy: 0.5323
Epoch 2/10
1749/1749 [==============================] - 5s 3ms/step - loss: 0.9469 - accuracy: 0.5409
Epoch 3/10
1749/1749 [==============================] - 5s 3ms/step - loss: 0.8754 - accuracy: 0.5489
Epoch 4/10
1749/1749 [==============================] - 5s 3ms/step - loss: 0.7522 - accuracy: 0.6615
Epoch 5/10
1749/1749 [==============================] - 5s 3ms/step - loss: 0.5686 - accuracy: 0.7753
Epoch 6/10
1749/1749 [==============================] - 5s 3ms/step - loss: 0.4507 - accuracy: 0.8353
Epoch 7/10
1749/1749 [==============================] - 5s 3ms/step - loss: 0.3334 - accuracy: 0.8754
Epoch 8/10
1749/1749 [==============================] - 5s 3ms/step - loss: 0.2469 - accuracy: 0.9182
Epoch 9/10
1749/1749 [==============================] - 5s 3ms/step - loss: 0.1891 - accuracy: 0.9383
Epoch 10/10
1749/1749 [==============================] - 5s 3ms/step - loss: 0.160

In [9]:

# Testing the model
print(colored("Testing the LSTM model", "green"))
score, accuracy = model.evaluate(test_tweets, pd.get_dummies(test_data['sent_score']).values, batch_size = 128)
print("Test accuracy: {}".format(accuracy))
print("Test loss: {}".format(1-accuracy))

Testing the LSTM model
517/517 [==============================] - 0s 758us/step
Test accuracy: 0.655705988407135
Test loss: 0.344294011592865
